In [7]:
# Step 1: Input text and chunking
input_text = """Berlin is the capital and largest city of Germany, both by area and by population.
Its more than 3.85 million inhabitants make it the European Union's most populous city, as measured by population within city limits.
The city is also one of the states of Germany and is the third smallest state in the country in terms of area.
Paris is the capital and most populous city of France.
It is situated along the Seine River in the north-central part of the country.
The city has a population of over 2.1 million residents within its administrative limits, making it one of Europe's major population centers."""

# Split into sentence-level chunks
test_chunks = [sentence.strip() for sentence in input_text.split('\n') if sentence.strip()]


In [8]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate

# Anthropic-style system prompt template
anthropic_contextual_retrieval_system_prompt = """<document>
{WHOLE_DOCUMENT}
</document>
Here is the chunk we want to situate within the whole document
<chunk>
{CHUNK_CONTENT}
</chunk>
Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else."""

# Create a basic prompt template
anthropic_prompt_template = PromptTemplate(
    input_variables=["WHOLE_DOCUMENT", "CHUNK_CONTENT"],
    template=anthropic_contextual_retrieval_system_prompt
)

# Wrap the prompt in HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate(prompt=anthropic_prompt_template)

# Final ChatPromptTemplate used for the model
anthropic_contextual_retrieval_final_prompt = ChatPromptTemplate(
    input_variables=["WHOLE_DOCUMENT", "CHUNK_CONTENT"],
    messages=[human_message_prompt]
)


In [9]:
import os
groq_api_key=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fe1af889b20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fe1af877f80>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_core.output_parsers import StrOutputParser

# Connect prompt, model, and parser into a chain
contextual_chunk_creation = anthropic_contextual_retrieval_final_prompt | llm | StrOutputParser()


In [11]:
contextual_chunks = []

for chunk in test_chunks:
    context = contextual_chunk_creation.invoke({
        "WHOLE_DOCUMENT": input_text,
        "CHUNK_CONTENT": chunk
    })
    contextual_chunks.append({
        "chunk": chunk,
        "context": context
    })

# Optional: Print to verify
for entry in contextual_chunks:
    print(f"Chunk: {entry['chunk']}\nContext: {entry['context']}\n{'-'*40}")


Chunk: Berlin is the capital and largest city of Germany, both by area and by population.
Context: Describing the population and size of Berlin.  

----------------------------------------
Chunk: Its more than 3.85 million inhabitants make it the European Union's most populous city, as measured by population within city limits.
Context: Describing Berlin's population and ranking within the EU.  

----------------------------------------
Chunk: The city is also one of the states of Germany and is the third smallest state in the country in terms of area.
Context: This sentence describes Berlin's status as a state within Germany.  

----------------------------------------
Chunk: Paris is the capital and most populous city of France.
Context: This chunk describes Paris as the capital and most populous city of France.  

----------------------------------------
Chunk: It is situated along the Seine River in the north-central part of the country.
Context: Describing the geographic location 

reranker simple


In [12]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")
query_embedding = model.encode("What is the population of Berlin?", convert_to_tensor=True)

# Score and sort by similarity
scored_chunks = []
for entry in contextual_chunks:
    chunk_embedding = model.encode(entry["chunk"], convert_to_tensor=True)
    score = util.pytorch_cos_sim(query_embedding, chunk_embedding).item()
    scored_chunks.append((entry, score))

# Sort by descending score
top_chunks = sorted(scored_chunks, key=lambda x: x[1], reverse=True)[:3]

# Print top reranked results
for (entry, score) in top_chunks:
    print(f"Score: {score:.4f}\nChunk: {entry['chunk']}\nContext: {entry['context']}\n{'='*50}")


/home/rishulgupta/contextualretrieverragprototype/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Score: 0.7287
Chunk: Berlin is the capital and largest city of Germany, both by area and by population.
Context: Describing the population and size of Berlin.  

Score: 0.5577
Chunk: Its more than 3.85 million inhabitants make it the European Union's most populous city, as measured by population within city limits.
Context: Describing Berlin's population and ranking within the EU.  

Score: 0.5454
Chunk: The city has a population of over 2.1 million residents within its administrative limits, making it one of Europe's major population centers.
Context: Describing Paris's population and significance. 



In [13]:
final_context = "\n".join([entry["chunk"] for (entry, _) in top_chunks])
final_prompt = f"""Based on the following context, answer the question:

Context:
{final_context}

Question:
What is the population of Berlin?"""

response = llm.invoke(final_prompt)
print("Final Answer:", response.content)


Final Answer: The context provides two figures for Berlin's population:

* **Over 3.85 million inhabitants** when considering the entire city area.
* **Over 2.1 million residents** within its administrative limits. 


So, the population of Berlin depends on how you define its boundaries. 


